In [12]:
import requests
import zipfile
import io
import os
import pandas as pd
from sqlalchemy.engine import create_engine
import openai 
from dotenv import load_dotenv
from IPython.display import display, Markdown
from pathlib import Path

In [13]:
class Prompter:
    def __init__(self, gpt_model):
        if not os.environ.get("OPENAI_API_KEY"):
            raise Exception("Please set the OPENAI_API_KEY environment variable")

        openai.api_key = os.environ.get("OPENAI_API_KEY")

        self.gpt_model = gpt_model

    def prompt_model_print(self, messages: list):
        response = openai.ChatCompletion.create(model=self.gpt_model, 
                                                messages=messages,
                                                temperature=0.2)
        display(Markdown(response["choices"][0]["message"]["content"]))
    
    def prompt_model_return(self, messages: list):
        response = openai.ChatCompletion.create(model=self.gpt_model, 
                                                messages=messages,
                                                temperature=0.2)
        return response["choices"][0]["message"]["content"]

In [14]:
# Get the current working directory
current_working_directory = os.getcwd()

# +
# Convert the current working directory to a Path object
script_dir = Path(current_working_directory)


predicted_data_path = '/Users/macpro/Documents/GitHub/fuel-electric-hybrid-vehicle-ml/data/predicted-data/vehicle_data_with_clusters.csv'

In [20]:
engine = create_engine("sqlite://")
df = pd.read_csv(predicted_data_path)

In [21]:
df.columns = df.columns.str.replace('.', '_')

# Replace the character '/' with '_per_' all entries
df.columns = df.columns.str.replace('/', '_per_')

# drop column hybrid_in_fuel	hybrid_in_electric	aggregate_levels	vehicle_type_cat
df = df.drop(['hybrid_in_fuel', 'hybrid_in_electric', 'aggregate_levels','transmission_','fuel_type'], axis=1)

/var/folders/2t/nqb9hcfs07n91h4v5p34slp00000gn/T/ipykernel_60992/32763826.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.replace('.', '_')


In [22]:
df.to_sql("vehicles", engine)

In [23]:
%load_ext sql
%sql engine

In [24]:
%%sql
SELECT *
FROM vehicles
LIMIT 3

*  sqlite://
Done.


index,vehicle_id,vehicleclass_,make_,model_1_,model_year,cylinders_,fuelconsumption_city(l_per_100km),fuelconsumption_hwy(l_per_100km),fuelconsumption_comb(l_per_100km),co2emissions_(g_per_km),number_of_gears,predicted_co2_rating,enginesize_(l),fuelconsumption_comb(mpg),smog_rating,transmission_type,mapped_fuel_type,type_of_wheel_drive,vehicle_type,motor_(kw),consumption_combinedle_per_100km,range1_(km),recharge_time(h),fuel_type2,range2_(km),hybrid_fuels,consumption_city(kwh_per_100km),fuelconsumption_hwy(kwh_per_100km),fuelconsumption_comb(kwh_per_100km),fuelconsumption_city(le_per_100km),fuelconsumption_hwy(le_per_100km),fuelconsumption_comb(le_per_100km),range_(km)
0,fuel-only_1,full-size,acura,integra,2023,4.0,7.9,6.3,7.2,167,7.0,6.0,1.5,39.0,7.0,continuously variable,premium gasoline,unspecified,fuel-only,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,fuel-only_2,full-size,acura,integra a-spec,2023,4.0,8.1,6.5,7.4,172,7.0,6.0,1.5,38.0,7.0,continuously variable,premium gasoline,unspecified,fuel-only,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,fuel-only_3,full-size,acura,integra a-spec,2023,4.0,8.9,6.5,7.8,181,6.0,6.0,1.5,36.0,6.0,manual,premium gasoline,unspecified,fuel-only,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
load_dotenv('.env')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [26]:
prompter = Prompter("gpt-3.5-turbo")


In [27]:
data_query = "Show hybrid vehicles"
sample_values = {df.columns[i]: df.values[0][i] for i in range(len(df.columns))}

datagen_prompts_2 = [
    {"role" : "system", "content" : "You are a data analyst specializing in SQL, you are presented with a natural language query, and you form queries to answer questions about the data."},
    {"role" : "user", "content" : f"Please generate 1 SQL queries for data with columns {', '.join(df.columns)} and sample values {sample_values}. \
                                    The table is called 'vehicles'. Use the natural language query {data_query}"},
]


prompter.prompt_model_print(datagen_prompts_2)

result1 = prompter.prompt_model_return(datagen_prompts_2)
result2 = prompter.prompt_model_return(datagen_prompts_2)

SELECT * FROM vehicles WHERE hybrid_fuels != '0' OR fuel_type2 != '0';

In [28]:
result1.split("\n\n")[0]

"SELECT * FROM vehicles WHERE hybrid_fuels != '0' OR fuel_type2 != '0';"

In [29]:
result2.split("\n\n")[0]

"SELECT * FROM vehicles WHERE hybrid_fuels != '0' OR fuel_type2 != '0';"

In [30]:
%sql {{result1.split("\n\n")[0]}}

*  sqlite://
Done.


index,vehicle_id,vehicleclass_,make_,model_1_,model_year,cylinders_,fuelconsumption_city(l_per_100km),fuelconsumption_hwy(l_per_100km),fuelconsumption_comb(l_per_100km),co2emissions_(g_per_km),number_of_gears,predicted_co2_rating,enginesize_(l),fuelconsumption_comb(mpg),smog_rating,transmission_type,mapped_fuel_type,type_of_wheel_drive,vehicle_type,motor_(kw),consumption_combinedle_per_100km,range1_(km),recharge_time(h),fuel_type2,range2_(km),hybrid_fuels,consumption_city(kwh_per_100km),fuelconsumption_hwy(kwh_per_100km),fuelconsumption_comb(kwh_per_100km),fuelconsumption_city(le_per_100km),fuelconsumption_hwy(le_per_100km),fuelconsumption_comb(le_per_100km),range_(km)
12408,hybrid_1,compact,chevrolet,volt,2012,4.0,6.7,5.9,6.4,54,0.0,10.0,1.4,0.0,7.0,continuously variable,premium gasoline,0,hybrid,111.0,2.5 (22.3 kWh/100 km),56.0,4.0,Z,550.0,electricity,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12409,hybrid_2,compact,chevrolet,volt,2013,4.0,6.7,5.9,6.4,45,0.0,10.0,1.4,0.0,7.0,continuously variable,premium gasoline,0,hybrid,111.0,2.4 (21.4 kWh/100 km),61.0,4.0,Z,550.0,electricity,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12410,hybrid_3,mid-size,ford,c-max energi,2013,4.0,5.8,6.5,6.1,80,0.0,10.0,2.0,0.0,8.0,continuously variable,regular gasoline,0,hybrid,35.0,2.7 ([23.2 kWh + 0.1 L]/100 km),32.0,2.5,X,856.0,electricity & regular gasoline,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12411,hybrid_4,mid-size,ford,fusion energi,2013,4.0,5.8,6.5,6.1,80,0.0,10.0,2.0,0.0,8.0,continuously variable,regular gasoline,0,hybrid,35.0,2.7 ([23.2 kWh + 0.1 L]/100 km),32.0,2.5,X,856.0,electricity & regular gasoline,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12412,hybrid_5,mid-size,toyota,prius plug-in hybrid,2013,4.0,4.7,4.8,4.7,101,0.0,10.0,1.8,0.0,8.0,continuously variable,regular gasoline,0,hybrid,60.0,2.5 ([18.0 kWh + 0.4 L]/100 km),18.0,1.5,X,845.0,electricity & regular gasoline,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12413,hybrid_6,compact,cadillac,elr,2014,4.0,7.6,6.7,7.2,50,0.0,10.0,1.4,0.0,8.0,continuously variable,premium gasoline,0,hybrid,117.0,2.9 (25.3 kWh/100 km),59.0,5.0,Z,493.0,electricity,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12414,hybrid_7,compact,chevrolet,volt,2014,4.0,6.7,5.9,6.4,45,0.0,10.0,1.4,0.0,7.0,continuously variable,premium gasoline,0,hybrid,111.0,2.4 (21.4 kWh/100 km),61.0,4.0,Z,550.0,electricity,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12415,hybrid_8,mid-size,ford,c-max energi,2014,4.0,5.8,6.5,6.1,80,0.0,10.0,2.0,0.0,8.0,continuously variable,regular gasoline,0,hybrid,35.0,2.7 ([23.2 kWh + 0.1 L]/100 km),32.0,2.5,X,856.0,electricity & regular gasoline,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12416,hybrid_9,mid-size,ford,fusion energi,2014,4.0,5.8,6.5,6.1,80,0.0,10.0,2.0,0.0,8.0,continuously variable,regular gasoline,0,hybrid,35.0,2.7 ([23.2 kWh + 0.1 L]/100 km),32.0,2.5,X,856.0,electricity & regular gasoline,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12417,hybrid_10,full-size,porsche,panamera s e-hybrid,2014,6.0,10.4,8.0,9.3,142,8.0,9.0,3.0,0.0,6.0,automatic,premium gasoline,0,hybrid,71.0,4.6 ([31.7 kWh + 1.0 L]/100 km),26.0,3.0,Z,871.0,electricity & premium gasoline,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
%sql {{result2.split("\n\n")[0]}}

*  sqlite://
Done.


index,vehicle_id,vehicleclass_,make_,model_1_,model_year,cylinders_,fuelconsumption_city(l_per_100km),fuelconsumption_hwy(l_per_100km),fuelconsumption_comb(l_per_100km),co2emissions_(g_per_km),number_of_gears,predicted_co2_rating,enginesize_(l),fuelconsumption_comb(mpg),smog_rating,transmission_type,mapped_fuel_type,type_of_wheel_drive,vehicle_type,motor_(kw),consumption_combinedle_per_100km,range1_(km),recharge_time(h),fuel_type2,range2_(km),hybrid_fuels,consumption_city(kwh_per_100km),fuelconsumption_hwy(kwh_per_100km),fuelconsumption_comb(kwh_per_100km),fuelconsumption_city(le_per_100km),fuelconsumption_hwy(le_per_100km),fuelconsumption_comb(le_per_100km),range_(km)
12408,hybrid_1,compact,chevrolet,volt,2012,4.0,6.7,5.9,6.4,54,0.0,10.0,1.4,0.0,7.0,continuously variable,premium gasoline,0,hybrid,111.0,2.5 (22.3 kWh/100 km),56.0,4.0,Z,550.0,electricity,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12409,hybrid_2,compact,chevrolet,volt,2013,4.0,6.7,5.9,6.4,45,0.0,10.0,1.4,0.0,7.0,continuously variable,premium gasoline,0,hybrid,111.0,2.4 (21.4 kWh/100 km),61.0,4.0,Z,550.0,electricity,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12410,hybrid_3,mid-size,ford,c-max energi,2013,4.0,5.8,6.5,6.1,80,0.0,10.0,2.0,0.0,8.0,continuously variable,regular gasoline,0,hybrid,35.0,2.7 ([23.2 kWh + 0.1 L]/100 km),32.0,2.5,X,856.0,electricity & regular gasoline,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12411,hybrid_4,mid-size,ford,fusion energi,2013,4.0,5.8,6.5,6.1,80,0.0,10.0,2.0,0.0,8.0,continuously variable,regular gasoline,0,hybrid,35.0,2.7 ([23.2 kWh + 0.1 L]/100 km),32.0,2.5,X,856.0,electricity & regular gasoline,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12412,hybrid_5,mid-size,toyota,prius plug-in hybrid,2013,4.0,4.7,4.8,4.7,101,0.0,10.0,1.8,0.0,8.0,continuously variable,regular gasoline,0,hybrid,60.0,2.5 ([18.0 kWh + 0.4 L]/100 km),18.0,1.5,X,845.0,electricity & regular gasoline,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12413,hybrid_6,compact,cadillac,elr,2014,4.0,7.6,6.7,7.2,50,0.0,10.0,1.4,0.0,8.0,continuously variable,premium gasoline,0,hybrid,117.0,2.9 (25.3 kWh/100 km),59.0,5.0,Z,493.0,electricity,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12414,hybrid_7,compact,chevrolet,volt,2014,4.0,6.7,5.9,6.4,45,0.0,10.0,1.4,0.0,7.0,continuously variable,premium gasoline,0,hybrid,111.0,2.4 (21.4 kWh/100 km),61.0,4.0,Z,550.0,electricity,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12415,hybrid_8,mid-size,ford,c-max energi,2014,4.0,5.8,6.5,6.1,80,0.0,10.0,2.0,0.0,8.0,continuously variable,regular gasoline,0,hybrid,35.0,2.7 ([23.2 kWh + 0.1 L]/100 km),32.0,2.5,X,856.0,electricity & regular gasoline,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12416,hybrid_9,mid-size,ford,fusion energi,2014,4.0,5.8,6.5,6.1,80,0.0,10.0,2.0,0.0,8.0,continuously variable,regular gasoline,0,hybrid,35.0,2.7 ([23.2 kWh + 0.1 L]/100 km),32.0,2.5,X,856.0,electricity & regular gasoline,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12417,hybrid_10,full-size,porsche,panamera s e-hybrid,2014,6.0,10.4,8.0,9.3,142,8.0,9.0,3.0,0.0,6.0,automatic,premium gasoline,0,hybrid,71.0,4.6 ([31.7 kWh + 1.0 L]/100 km),26.0,3.0,Z,871.0,electricity & premium gasoline,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df